In [1]:
%env TF_CPP_MIN_LOG_LEVEL=3

import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import datetime
from pathlib import Path

import plotly.express as px

import tensorflow as tf

from tqdm.notebook import tqdm

from importlib import reload
import keras_model_creator
import sentinel_utils
import plot_utils
from data_generator import DataGenerator

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
# sentinel_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12']
# shards_dir = Path.home().joinpath('sentinel_data', 'shards')
# indices = [sentinel_bands.index(b) for b in ['B3', 'B8', 'B6', 'B11']]
# shards = list(shards_dir.joinpath('features_201706').glob('feature*.npy'))
# for shard in tqdm(shards):
#     data = np.copy(np.load(shard))
#     np.save(
#         shards_dir.joinpath('features_2017', shard.stem),
#         data[..., indices]
#     )

In [3]:
reload(sentinel_utils)

model_dir = Path('models', 'selected_model')
shards_dir = Path.home().joinpath('sentinel_data', 'shards')

utils = sentinel_utils.SentinelUtils(min_occurrences=20000)

selected_classes = utils.get_processed_labels()

data_summary = utils.get_data_summary(
    shards_dir, selected_classes, overwrite_existing=False
)

loss = 'binary_crossentropy'
batch_size = 64
base_filters = 32

model_parent_dir = Path('models')
model_dir = model_parent_dir.joinpath(
    f'{loss}-{len(selected_classes.index)}'
    f'-{selected_classes.shape[1]}-{batch_size}-{base_filters}'
)
model_dir.mkdir(parents=True, exist_ok=True)

params = dict(
    loss=loss,
    base_filters=base_filters,
    shards_dir=shards_dir,
    selected_classes=selected_classes,
    data_summary=data_summary,
    batch_size=64,
    dropout=0.3,
    epochs=10,
    overwrite=True,
    verbose=1,
    print_log=0
)

In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=1e-4,
    beta_1=0.9,
    beta_2=0.999,
)
params['opt'] = opt
params['model_dir'] = model_dir

reload(keras_model_creator)
model, testing_generator = keras_model_creator.KerasModelCreator(**params).run()

Epoch 1/10


I0000 00:00:1722597184.362828   32421 service.cc:145] XLA service 0x7f823800a780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722597184.362869   32421 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1722597192.312522   32421 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3550/3550 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4059 - auc: 0.7907 - loss: 0.4629 - macrof1score: 0.4145 - microf1score: 0.5332 - prc: 0.5807 - precision: 0.5877 - recall: 0.4885 - weightedf1score: 0.5326
Epoch 1: val_recall improved from 0.50000 to 0.54220, saving model to models/binary_crossentropy-237212-7-64-32/model.keras
3550/3550 ━━━━━━━━━━━━━━━━━━━━ 146s 38ms/step - accuracy: 0.4059 - auc: 0.7907 - loss: 0.4629 - macrof1score: 0.4145 - microf1score: 0.5332 - prc: 0.5807 - precision: 0.5878 - recall: 0.4885 - weightedf1score: 0.5326 - val_accuracy: 0.4597 - val_auc: 0.8688 - val_loss: 0.3796 - val_macrof1score: 0.4241 - val_microf1score: 0.6069 - val_prc: 0.6919 - val_precision: 0.6892 - val_recall: 0.5422 - val_weightedf1score: 0.5702 - learning_rate: 1.0000e-04
Epoch 2/10
2602/3550 ━━━━━━━━━━━━━━━━━━━━ 33s 36ms/step - accuracy: 0.4725 - auc: 0.8868 - loss: 0.3482 - macrof1score: 0.4438 - microf1score: 0.6386 - prc: 0.7351 - precision: 0.7336 - recall: 0.5653 - weight

In [ ]:
model.summary()